To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction

After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

In [1]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=8000, num_dims=256)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("/home/rs/21CS91R01/research/fixed-length-fingerprint-extractors/flx/pre_trained_model") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from /home/rs/21CS91R01/research/fixed-length-fingerprint-extractors/flx/pre_trained_model/best_model.pyt


/home/rs/21CS91R01/anaconda3/envs/flx/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Now we need to specify the dataset, for which we want to extract the embeddings

In [2]:
import os

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath("example-dataset")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 10 subjects and a total of 100 samples.


  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:34<00:00,  8.52s/it]


## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [6]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 10
benchmark = create_verification_benchmark(
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

100%|██████████| 10/10 [00:00<00:00, 13635.58it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [7]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
print(minutia_embeddings._array)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
[[ 0.06483774  0.00975441  0.04458646 ...  0.01890967  0.08077914
   0.01240349]
 [ 0.03562802  0.01733589  0.0456029  ...  0.01502696  0.08776784
  -0.00227975]
 [ 0.04463532  0.02813711  0.03865627 ...  0.00685132  0.08758652
   0.01492492]
 ...
 [ 0.05595645 -0.03505693  0.10371689 ...  0.102452    0.08917881
   0.17647512]
 [ 0.04721681 -0.0290929   0.10208201 ...  0.11360291  0.0901756
   0.14846209]
 [ 0.04137497 -0.02779733  0.07993095 ...  0.11635151  0.09079733
   0.17041668]]
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.
Created IdentifierSet with 10 subjects and a total of 100 samples.


100%|██████████| 1450/1450 [00:00<00:00, 75181.61it/s]

Equal-Error-Rate: 0.0


To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)

In [8]:
from flx.visualization.plot_DET_curve import plot_verification_results

figure_path = "DET_curve"

# Lists are used to allow for multiple models to be plotted in the same figure
plot_verification_results(figure_path, results=[results], model_labels=["DeepPrint_TexMinu"], plot_title="example-dataset - verification")

ImportError: cannot import name 'common_texification' from 'matplotlib.backends.backend_pgf' (/home/rs/21CS91R01/anaconda3/envs/flx/lib/python3.10/site-packages/matplotlib/backends/backend_pgf.py)